In [99]:
import argparse
import os
import time
import math
import numpy as np
import pylab
import scipy.stats
import matplotlib
matplotlib.rc('mathtext', fontset='stixsans', default='regular')
import re
import rmgpy
from rmgpy.quantity import constants
from rmgpy.kinetics import Arrhenius, ArrheniusEP, KineticsData
from rmgpy.data.base import getAllCombinations
from autotst.database import *
from rmgpy.species import Species
from rmgpy.data.rmg import RMGDatabase
import logging
from collections import defaultdict, OrderedDict
import pandas as pd
import itertools
import IPython
from IPython.display import display, Markdown
def mprint(s): display(Markdown(s))
import cPickle as pickle
# attempt at making the cells wider:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
"""import os
from collections import defaultdict, OrderedDict
import re"""

In [100]:
def get_csv(path):
    add_df = pd.DataFrame.from_csv(path)

    results = defaultdict(OrderedDict)
    for i, entry in enumerate(add_df.index):
        r = OrderedDict()

        label = add_df.T[entry].name
        reactants, products = label.split('_')
        r1, r2 = reactants.split('+')
        p1, p2 = products.split('+')

        #label = label.replace('+', ' + ')
        #label = label.replace('_', ' <=> ')

        #r['label'] = label

        r['species'] = [r1, r2, p1, p2]
        #print r['species']
        assert len(r['species']) == 4
        r['d12'] = add_df.T[entry]['d12']
        r['d13'] = add_df.T[entry]['d13']
        r['d23'] = add_df.T[entry]['d23']
        assert r['d12'] > 0
        assert r['d13'] > 0
        assert r['d23'] > 0

        #r['method'] = 'm062x/6-311+G(2df,2p)'
        #r['older reaction data'] = np.nan
        #r['index'] = np.nan
        #r['degeneracy'] = 1
        #r['rank'] = np.nan
        #r['shortDesc'] = 'M06-2X/6-311+G(2df,2p) calculation via group additive TS generator.'

        results[i] = r

    return pd.DataFrame(results)

In [111]:
def get_need_to_add(csv_df, known_species):
    found_species = {}
    need_to_add = []

    for i, row in csv_df.T.iterrows():

        r1, r2, p1, p2 = row['species']

        mr1 = Molecule(SMILES = r1)
        mr2 = Molecule(SMILES = r2)
        mp1 = Molecule(SMILES = p1)
        mp2 = Molecule(SMILES = p2)

        reaction = Reaction(reactants = [mr1, mr2],
                            products = [mp1, mp2],
                            degeneracy = 1,
                            duplicate = False,
                            reversible = True)

        relavent_species = [mr1, mr2, mp1, mp2]
        relavent_labels = {}

        for rel_species in relavent_species:
            for label in known_species:
                known_spec = known_species[label]
                if known_spec.isIsomorphic(rel_species):
                    found_species[rel_species] = label
                    relavent_labels[rel_species] = label

            try:
                a = found_species[rel_species]
            except:
                need_to_add.append(rel_species.toSMILES())
                relavent_labels[rel_species] = '****'
                #logging.warning('{} is missing from species dictionary'.format(rel_species))

        """lr1 = relavent_labels[mr1]
        lr2 = relavent_labels[mr2]
        lp1 = relavent_labels[mp1]
        lp2 = relavent_labels[mp2]

        Label = '{} + {} <=> {} + {}'.format(lr1, lr2, lp1, lp2)
        assert Label is not None
        assert Label is not ''
        print Label"""

    need_to_add = list(set(need_to_add))
    
    return need_to_add

In [131]:
def get_updated_dictionary_entries(load_path, need_to_add):
    
    library = rmgpy.data.kinetics.library.KineticsLibrary()
    pattern = True
    library.loadOldDictionary(load_path, pattern=pattern)

    print 'Starting Species:', len(library.entries.values()) 

    for i, entry in enumerate(library.entries.values()):
        entry.index = i
        library.entries[entry.index] = entry
        del library.entries[entry.label]


    i = i+2

    for j, species in enumerate(need_to_add):
        mole = Molecule(SMILES = species)

        adjlist = mole.toAdjacencyList()
        multiplicity = None
        multiplicity =  adjlist[adjlist.find("multiplicity ")+13:adjlist.find("multiplicity ")+14]
        if multiplicity is not None:
            adjlist = re.sub(r'multiplicity .*', 'multiplicity [{}]'.format(multiplicity), adjlist)

        group = rmgpy.molecule.group.Group()
        group.fromAdjacencyList(adjlist)

        atom_counts = {}
        rel_label = ''
        for atom in ['C', 'H', 'O']:
            atom_counts[atom] = species.count(atom)
            if species.count(atom) > 0:
                rel_label = rel_label + atom + str(species.count(atom))

        new_ID = None
        max_ID = -1
        for entry in library.entries.values():
            existing_label = entry.label

            if rel_label not in existing_label:
                continue

            if rel_label == existing_label:
                # Atleast one, but not necessarily more existing with same label
                max_ID = 0

            #print rel_label, ' : ', existing_label
            if existing_label.find('-') > 0:
                ID_str = existing_label[existing_label.find('-')+1:]
                ID = int(ID_str)
                existing_label = existing_label[:existing_label.find('-')]

                if existing_label == rel_label and ID > max_ID:
                    # Multiple exisitng labels
                    max_ID = ID

        if max_ID > -1:
            #All with existing labels
            new_ID = max_ID + 1
            rel_label = rel_label + '-' + str(new_ID)

        #print '\t', rel_label, ':', max_ID


        """
        3 Scenerios:
        No existing -> no need for ID number
        Only one existing -> needs to have ID of 1
        Multiple existing -> needs to have a unique ID
        """

        library.loadEntry(i+j,
                          rel_label,
                          None,
                          degeneracy=1,
                          duplicate=False,
                          reversible=True,
                          reference=None,
                          referenceType='',
                          shortDesc='',
                          longDesc='',
                          has_pdep_route=False,
                          )

        library.entries[i+j].item = group

        """library.entries[label] = Entry(index = i+j,
                                       label = rel_label,
                                       item = group,
                                       data = None,
                                       reference = None,
                                       referenceType = '',
                                       shortDesc = '',
                                       longDesc = ''.strip(),
                                      )"""
    entry_labels = [entry.label for entry in library.entries.values()]
    assert len(entry_labels) == len(list(set(entry_labels))), 'Non-unique labels in dictionary'

    print 'Final Species:', len(library.entries)
    return library.entries.values()

In [132]:
def ugly_save_dictionary(path, entries):
    f = open(path, 'w')
    for entry in entries:
        multiplicity = entry.item.multiplicity
        adjlist = entry.item.toAdjacencyList()
        if multiplicity is not None:
            #adjlist = re.sub(r'multiplicity .*', 'multiplicity {}'.format(multiplicity), adjlist)
            adjlist = re.sub('\[', '', adjlist)
            adjlist = re.sub('\]', '', adjlist)
        f.write(entry.label)
        f.write('\n')
        f.write(adjlist)
        f.write('\n')
                
    f.close()
    print 'Saved new dictionary'
    return

In [133]:
def update_reactions():
    from rmgpy.data.base import *
    from autotst.database import DistanceData
    r_db = TransitionStateDepository()
    path = os.path.join(os.getcwd(), 'database/H_Abstraction/TS_training/reactions.py')
    local_context = {'DistanceData': DistanceData, "nan":np.nan, "entry": Entry}
    r_db.load(path, local_context=local_context)
    # Remove above
    # Remove above
    # Remove above
    # Remove above
    # Remove above
    # Remove above
    # Remove above
    # Remove above
    # Remove above
    known_species = rmgpy.data.base.Database().getSpecies("./database/H_Abstraction/TS_training/dictionary.txt")

    found_species = {}
    need_to_add = []

    Index = 0
    for entry in r_db.entries.values():
        if Index < entry.index:
            Index = entry.index
    Index = Index + 1

    print Index
    print

    for i, row in csv_df.T.iterrows():

        r1, r2, p1, p2 = row['species']

        mr1 = Molecule(SMILES = r1)
        mr2 = Molecule(SMILES = r2)
        mp1 = Molecule(SMILES = p1)
        mp2 = Molecule(SMILES = p2)

        reaction = Reaction(reactants = [mr1, mr2],
                            products = [mp1, mp2],
                            degeneracy = 1,
                            duplicate = False,
                            reversible = True)


        Distances = {'d12':row['d12'], 'd13':row['d13'], 'd23':row['d23']}
        Method = 'm062x/6-311+G(2df,2p)'
        distance_data = DistanceData(distances = Distances, method = Method)

        """entry = Entry(index = Index + i,
                      label = 'prob not', #TODO Need to add back in
                      item = reaction,
                      data = distance_data, #TODO Need to add back in
                      reference = None,
                      referenceType = '',
                      shortDesc = '',
                      longDesc = ''.strip(),
                      rank = None,
                     )"""

        Label = 'Please fix me'
        ShortDesc = 'M06-2X/6-311+G(2df,2p) calculation via group additive TS generator.'


        mr1 = Molecule(SMILES = r1)
        mr2 = Molecule(SMILES = r2)
        mp1 = Molecule(SMILES = p1)
        mp2 = Molecule(SMILES = p2)

        relavent_species = [mr1, mr2, mp1, mp2]
        relavent_labels = {}

        for rel_species in relavent_species:
            for label in known_species:
                known_spec = known_species[label]
                if known_spec.isIsomorphic(rel_species):
                    found_species[rel_species] = label
                    relavent_labels[rel_species] = label

            try:
                a = found_species[rel_species]
            except:
                need_to_add.append(rel_species.toSMILES())
                relavent_labels[rel_species] = '****'
                logging.warning('{} is missing from species dictionary'.format(rel_species))

        lr1 = relavent_labels[mr1]
        lr2 = relavent_labels[mr2]
        lp1 = relavent_labels[mp1]
        lp2 = relavent_labels[mp2]

        Label = '{} + {} <=> {} + {}'.format(lr1, lr2, lp1, lp2)
        print Label


        r_db.loadEntry(Index + i,
                      reactant1=None,
                      reactant2=None,
                      reactant3=None,
                      product1=None,
                      product2=None,
                      product3=None,
                      distances = distance_data,
                      degeneracy=1,
                      label = Label,
                      duplicate=False,
                      reversible=True,
                      reference=None,
                      referenceType = '',
                      shortDesc = ShortDesc,
                      longDesc = '',
                      rank=None,
                      )


        r_db.entries['{0:d}:{1}'.format(Index + i, Label)].item = reaction

        #print
        #print 'here'
        #print entry.index
        #print entry.label
        #print entry.item.degeneracy
        #print entry.rank
        #print entry.data.method
        #print entry.shortDesc

    need_to_add = list(set(need_to_add))

<ipython-input-133-b145513cdca5>:1: SyntaxWarning: import * only allowed at module level
  def update_reactions():


In [134]:
#csv_path = os.path.join(os.path.expandvars('$RMGpy'), "..",  'AutoTST')

csv_path = 'distance_data.csv'
dict_path = 'database/H_Abstraction/TS_training/dictionary.txt'
old_style_dict_path = 'database/H_Abstraction/TS_training/old_dictionary.txt'
new_dict_path = 'updated_dictionary.txt'

csv_df = get_csv(csv_path)
print 'New Reactions: ',csv_df.shape[1]

known_species = rmgpy.data.base.Database().getSpecies(dict_path)
need_to_add = get_need_to_add(csv_df=csv_df, known_species=known_species)
print 'New Species: ', len(need_to_add)

all_dict_entries = get_updated_dictionary_entries(old_style_dict_path, need_to_add)
ugly_save_dictionary(new_dict_path, all_dict_entries)

known_species = rmgpy.data.base.Database().getSpecies(new_dict_path)




New Reactions:  920
New Species:  985
Starting Species: 282
Final Species: 1267
Saved new dictionary



# TODO Update Dictionary

In [ ]:
db = rmgpy.data.base.Database()
x = this_is_mine(db)

path = os.path.join(os.getcwd(), 'database/H_Abstraction/TS_training/old_dictionary.txt')
pattern = True

x.loadOldDictionary(path, pattern=pattern)

x.database.entries.values()

In [ ]:
reactants = [Molecule(SMILES="CCCC"), Molecule(SMILES="[O]O")]
products = [Molecule(SMILES="[CH2]CCC"), Molecule(SMILES="OO")]
rmg_reaction = Reaction(reactants=reactants, products=products)
rmg_reaction.degeneracy

print rmg_reaction

global_context = { '__builtins__': None }

from autotst.database import DistanceData
local_context = {"entry": Entry, 'DistanceData': DistanceData, "nan":np.nan}
r_db = TransitionStateDepository()
path = os.path.join(os.getcwd(), 'database/H_Abstraction/TS_training/reactions.py')
r_db.load(path, local_context=local_context)
#r_db.loadEntry
#r_db.entries

In [ ]:
#path = os.path.join(os.path.expandvars('$RMGpy'), "..",  'AutoTST', 'database')
#path = os.path.join(path, 'H_Abstraction', 'TS_training', 'reactions.py')

path = os.path.join(os.getcwd(), 'database/H_Abstraction/TS_training/reactions.py')

r_df = get_existing_reactions(path)
r_df.shape

In [ ]:
r_df.T.dropna(subset = ['label']).T
r_df.drop_duplicates()
r_df.shape

In [ ]:
r_df

In [ ]:
csv_df

In [ ]:
r_db.s

In [ ]:
print 'Found:', len(found_species)
print 'Need to add:', len(need_to_add)
print len(known_species)

In [ ]:
for entry in r_db.entries.values():
    if isinstance(entry.item, Reaction):
        #Write out additional data if depository or library
        #kinetic rules would have a Group object for its reactants instead of Species
        try:
            if isinstance(entry.item.reactants[0], Species):
                # Add degeneracy if the reaction is coming from a depository or kinetics library
                #print '    degeneracy = {0:.1f},\n'.format(entry.item.degeneracy)
                x = 1
        except:
            print entry
            print entry.index
            print entry.item
            print entry.item.reactants
            break

r_db.save('test_saving')
print

In [ ]:
found_species = {}
need_to_add = []
for specieseseses in csv_df.T['label']:
    for species in specieseseses:
        mole = Molecule(SMILES = species)
        
        for label in known_species:
            known_spec = known_species[label]
            if known_spec.isIsomorphic(mole):
                found_species[mole] = label
        
        try:
            a = found_species[mole]
        except:
            need_to_add.append(mole)



In [ ]:
print 'Found:', len(found_species)
print 'Need to add:', len(need_to_add)

In [ ]:
def trim_DF(DF):
    #print DF.shape

    #print '\t dropping NA Reactions'
    DF = DF.T.dropna(subset = ['label']).T
    
    #print '\t dropping NA Distances'
    DF = DF.T.dropna(subset = ['d12']).T
    DF = DF.T.dropna(subset = ['d13']).T
    DF = DF.T.dropna(subset = ['d23']).T
    #print DF.shape

    #print '\t dropping Column Duplicates'
    DF = DF.T.drop_duplicates().T
    #print DF.shape

    
    #print '\t dropping Reaction Duplicates'
    
    #   sorting priorities: older > index
    DF = DF.T.sort_values(['older reaction data', 'index']).T
    DF = DF.T.drop_duplicates(['label'], keep = 'first').T
    #print DF.shape

    return DF

In [ ]:
print '\tOld:'
print r_df.count(axis = 1)
print
print '\tAdditional:'
print csv_df.count(axis = 1)

DF = pd.concat([r_df, csv_df], axis = 1, ignore_index = True)
print
print '\tBefore Trimming:'
print DF.count(axis = 1)
DF = trim_DF(DF)

for index in DF.T.index:
    DF[index]['index'] = index + 1

print
print '\tAfter Trimming:'
print DF.count(axis = 1)
DF

In [ ]:
def get_existing_reactions(path):
    global_context = { '__builtins__': None }

    from autotst.database import DistanceData
    local_context = {'DistanceData': DistanceData, "nan":np.nan}
    r_db = TransitionStateDepository()
    r_db.load(path, local_context=local_context)

    results = defaultdict(OrderedDict)

    for i, entry in enumerate(r_db.entries.values()):
        r = OrderedDict()
        r['index'] = entry.index
        r['label'] = entry.label
        r['degeneracy'] = entry.item.degeneracy
        r['rank'] = entry.rank
        r['method'] = entry.data.method
        r['shortdesc'] = entry.shortDesc
        r['older reaction data'] = 1

        for key in entry.data.distances:
            r[key] = entry.data.distances[key]

        results[i] = r

    reactions_df = pd.DataFrame(results)
    return reactions_df

In [ ]:
for index in range(DF.shape[1]-1):
        new_entry = rmgpy.data.base.Entry()
        
        new_entry.index = DF[index]['index']
        new_entry.label = DF[index]['label']
        #degeneracy = DF[index]['degeneracy']
        reactants, products = DF[index]['label'].split(' <=> ')
        print reactants
        print products
        r1, r2 = reactants.split(' + ')
        p1, p2 = products.split(' + ')
        #TODO Need to put reactanst and products into smiles
        """reactants = [Molecule(SMILES=r1), Molecule(SMILES=r2)]
        products = [Molecule(SMILES=p1), Molecule(SMILES=p2)]
        rmg_reaction = Reaction(reactants=reactants, products=products)
        degeneracy = rmg_reaction.degeneracy"""
        
        d12 = DF[index]['d12']
        d13 = DF[index]['d13']
        d23 = DF[index]['d23']
        
        print new_entry.data
        break
        new_entry.data.method = DF[index]['method']
        new_entry.data.distances = {'d12':d12, 'd13':d13, 'd23':d23}
        
        rank = DF[index]['rank']
        shortDesc = DF[index]['shortDesc']
        
        break

In [ ]:
def ugly_write_new_reactions(path):
    entries = []
    for index in range(DF.shape[1]-1):
        my_list = []
        index = DF[index]['index']
        label = DF[index]['label']
        degeneracy = DF[index]['degeneracy']
        d12 = DF[index]['d12']
        d13 = DF[index]['d13']
        d23 = DF[index]['d23']
        method = DF[index]['method']
        rank = DF[index]['rank']
        shortDesc = DF[index]['shortDesc']

        if rank != np.nan:
            #Two options for when rank is not specified and when it is
            #Ugly but wrote it as list to be a little easier to understand rather than large string with formatting
            my_list = ['entry(\n\tindex = ',
                       str(index),
                       ',\n\tlabel = \"',
                       label,
                       '\",\n\tdegeneracy = ',
                       str(degeneracy),
                       ',\n\tdistances = DistanceData(\n\t\tdistances = {\'d12\': ',
                       str(d12),
                       ', \'d13\': ',
                       str(d13),
                       ', \'d23\': ',
                       str(d23),
                       '},\n\t\tmethod = \'',
                       method,
                       '\',\n\t),\n\trank = ',
                       str(rank),
                       ',\n\tshortDesc = u\"\"\"',
                       shortDesc,
                       '\"\"\",\n)'
                      ]
        else:
            my_list = ['entry(\n\tindex = ',
                       str(index),
                       ',\n\tlabel = \"',
                       label,
                       '\",\n\tdegeneracy = ',
                       str(degeneracy),
                       ',\n\tdistances = DistanceData(\n\t\tdistances = {\'d12\': ',
                       str(d12),
                       ', \'d13\': ',
                       str(d13),
                       ', \'d23\': ',
                       str(d23),
                       '},\n\t\tmethod = \'',
                       method,
                       '\',\n\t),\n\tshortDesc = u\"\"\"',
                       shortDesc,
                       '\"\"\",\n)'
                      ]

        entries.append(''.join(my_list))

    f = open(path, 'w')
    f.write('\n\n'.join(entries))
    f.close()
    return

In [ ]:
path = 'updated_reactions.py'
ugly_write_new_reactions(path)

# Dictionary Update
#


In [ ]:
print os.getcwd()

In [ ]:
global_context = { '__builtins__': None }

path = os.path.join(os.getcwd(), 'database/H_Abstraction/TS_training/reactions.py')
path = os.path.join(os.getcwd(), 'updater_test/reactions.py')
from autotst.database import DistanceData
local_context = {'DistanceData': DistanceData, "nan":np.nan}
r_db = TransitionStateDepository()
r_db.load(path, local_context=local_context)
r_db.entries

In [ ]:
blem = rmgpy.data.base.Database()
blem.entries = blem.getSpecies("./database/H_Abstraction/TS_training/dictionary.txt")
blem.entries.values()[0]

In [ ]:
known_species = rmgpy.data.base.Database().getSpecies("./database/H_Abstraction/TS_training/dictionary.txt")
known_species

#print known_species.values()
m1 = Molecule(SMILES="[CH2]C(C)CO")
relavent_labels = {}
for label in known_species:
    species = known_species[label]
    if species.isIsomorphic(m1):
        relavent_labels[m1] = label
        break
relavent_labels

In [ ]:
known_species = rmgpy.data.base.Database().getSpecies("./database/H_Abstraction/TS_training/dictionary.txt")
p_count = 0
s_count = 0

to_add = []
total_species = []

for reaction in csv_df.T['label']:
    #print reaction
    r, p = reaction.split(' <=> ')
    r1, r2 = r.split(' + ')
    p1, p2 = p.split(' + ')
    #print r1, r2, p1, p2
    involved_species = [Molecule(SMILES=r1),
                        Molecule(SMILES=r2),
                        Molecule(SMILES=p1),
                        Molecule(SMILES=p2)]
    
    for a in involved_species:
        c = False
        for b in total_species:
            if b.isIsomorphic(a):
                c = True
        if not c:
            to_add.append(a)
    
    for new_species in involved_species:
        matches_one = False
        for old_species in known_species.values():
            if old_species.isIsomorphic(new_species):
                matches_one = True
        
        if matches_one:
            p_count += 1
        else:
            to_add[new_species] = new_species.toAdjacencyList()
            s_count += 1
            
print p_count
print s_count

print
print len(known_species)
print len(to_add)
print len(list(set(to_add.keys())))
print
to_add.keys()

In [ ]:
total_sp = []

for reaction in csv_df.T['label']:
    #print reaction
    r, p = reaction.split(' <=> ')
    r1, r2 = r.split(' + ')
    p1, p2 = p.split(' + ')
    
    sp_list = [r1, r2, p1, p2]
    print reaction
    for sp in sp_list:
        total_sp.append(sp)
        
print len(total_sp)/4
total_sp = list(set(total_sp))
print len(total_sp)

In [ ]:
r1, r2 = entry.split("_")[0].split("+")
m1 = Molecule(SMILES=r1)
m2 = Molecule(SMILES=r2)
combined = Molecule.merge(m1,m2)
combined.

In [ ]:
t = RMGDatabase()
t

In [ ]:
blem = rmgpy.data.base.Database()
blem.getSpecies("./database/H_Abstraction/TS_training/dictionary.txt")


In [ ]:
blem.entries = OrderedDict({Molecule(SMILES="CCC"): e})
blem.saveDictionary("test.py")

In [ ]:
#e = rmgpy.data.base.Entry(label="CCC", item=Molecule(SMILES="CCC").toAdjacencyList())
print Molecule(SMILES="CCC").toAdjacencyList()

In [ ]:
rmg_database = RMGDatabase()
database_path = os.path.join(os.path.expandvars('$RMGpy'), "..",  'RMG-database', 'input')
rmg_database.load(database_path,
                 kineticsFamilies=['H_Abstraction'],
                 transportLibraries=[],
                 reactionLibraries=[],
                 seedMechanisms=[],
                 thermoLibraries=['primaryThermoLibrary', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                 solvation=False,
                 )

# TODO: Edit this so it works with multiple databases

ts_database = TransitionStates()
path = os.path.join(os.path.expandvars("$RMGpy"), "..", "AutoTST", "database", "H_Abstraction")
global_context = { '__builtins__': None }
local_context={'DistanceData': DistanceData}
family = rmg_database.kinetics.families["H_Abstraction"]
ts_database.family = family
ts_database.load(path, local_context, global_context)

In [7]:
class this_is_mine:
    def __init__(self, database):
        self.database = database
    
    def loadOldDictionary(self, path, pattern):
            """
            Parse an old-style RMG database dictionary located at `path`. An RMG
            dictionary is a list of key-value pairs of a one-line string key and a
            multi-line string value. Each record is separated by at least one empty
            line. Returns a ``dict`` object with the values converted to
            :class:`Molecule` or :class:`Group` objects depending on the
            value of `pattern`.
            """

            # The dictionary being loaded
            self.database.entries = {}
            # The current record
            record = ''

            fdict=None
            # Process the dictionary file
            try:
                fdict = open(path, 'r')
                for line in fdict:
                    line = line.strip()
                    # If at blank line, end of record has been found
                    if len(line) == 0 and len(record) > 0:
                        # Label is first line of record
                        lines = record.splitlines()
                        label = lines[0]
                        # Add record to dictionary
                        self.database.entries[label] = Entry(label=label, item=record)
                        # Clear record in preparation for next iteration
                        record = ''
                    # Otherwise append line to record (if not empty and not a comment line)
                    else:
                        line = removeCommentFromLine(line).strip()
                        if len(line) > 0:
                            record += line + '\n'
                # process the last record! (after end of for loop)
                # Label is first line of record
                if record:
                    label = record.splitlines()[0]
                    # Add record to dictionary
                    self.database.entries[label] = Entry(label=label, item=record)
            except DatabaseError, e:
                logging.exception(str(e))
                raise
            except IOError, e:
                logging.exception('Database dictionary file "' + e.filename + '" not found.')
                raise
            finally:
                if fdict: fdict.close()

            # Convert the records in the dictionary to Molecule, Group, or
            # logical objects
            try:
                for label in self.database.entries:
                    record = self.database.entries[label].item
                    lines = record.splitlines()
                    # If record is a logical node, make it into one.
                    if re.match("(?i)\s*(NOT\s)?\s*(OR|AND|UNION)\s*(\{.*\})", lines[1]):
                        self.database.entries[label].item = makeLogicNode(' '.join(lines[1:]) )
                    # Otherwise convert adjacency list to molecule or pattern
                    elif pattern:
                        self.database.entries[label].item = Group().fromAdjacencyList(record)
                    else:
                        self.database.entries[label].item = Molecule().fromAdjacencyList(record,saturateH=True)
            except InvalidAdjacencyListError, e:
                logging.error('Error while loading old-style dictionary "{0}"'.format(path))
                logging.error('Error occurred while parsing adjacency list "{0}"'.format(label))
                raise
